# Day 3: Gear Ratios

## Part 1

> <p>You and the Elf eventually reach a <a href="https://en.wikipedia.org/wiki/Gondola_lift" target="_blank">gondola lift</a> station; he says the gondola lift will take you up to the <em>water source</em>, but this is as far as he can bring you. You go inside.</p>
> <p>It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.</p>
> <p>"Aaah!"</p>
> <p>You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.</p>
> <p>The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can <em>add up all the part numbers</em> in the engine schematic, it should be easy to work out which part is missing.</p>
> <p>The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently <em>any number adjacent to a symbol</em>, even diagonally, is a "part number" and should be included in your sum. (Periods (<code>.</code>) do not count as a symbol.)</p>
> <p>Here is an example engine schematic:</p>
> <pre><code>467..114..
> ...*......
> ..35..633.
> ......#...
> 617*......
> .....+.58.
> ..592.....
> ......755.
> ...$.*....
> .664.598..
> </code></pre>


In [ ]:
from re import Match

from advent23 import Checker, disp_name, get_chk, get_inp
from advent23.blake import remap

chk = get_chk()
inp = get_inp(3)

NULL = "."
"""Character representing "not a symbol" in the input."""

SYMBOLS = "#$%&*+-/=@"
"""Possible symbols in the input.

Found by taking the intersection of symbols in `string.punctuation` and
`input/blake.txt`, excluding `.` (period)."""

RE_SYMBOLS = r"#\$%&\*\+\-\/=@"
"""Same as `SYMBOLS`, but with special regex characters escaped.

If you pursue a regex solution, forgetting to escape these characters will introduce
subtle bugs.
"""

check = Checker(chk)
checks = {}

#### Part 1

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..



#### Part 2

<INPUT UNKNOWN>


In [ ]:
width = inp["a"].find("\n")
oneline = inp["a"].replace("\n", "")


def get_pos(sym: Match[str]) -> tuple[int, int, int]:
    start, end = sym.span()
    end -= 1
    row, lcol = divmod(start, width)
    rcol = lcol + end - start
    return row, lcol, rcol

In [ ]:
symbols = RE_SYMBOLS.replace("$", "$$")

symbol_positions = [
    get_pos(sym)[:2] for sym in remap(r=rf"[{symbols}]").finditer(oneline)
]
chk["symbol_positions"] = [list(pos) for pos in symbol_positions]

#### Symbol positions

[[1,
 12],
 [1,
 18],
 [1,
 30],
 [1,
 34],
 [1,
 42],
 [1,
 69],
 [1,
 82],
 [1,
... <view truncated> ...


In [ ]:
chk["not_parts"] = [114, 58]

#### Not parts

[114, 58]

> <p>In this schematic, two numbers are <em>not</em> part numbers because they are not adjacent to a symbol: <code>114</code> (top right) and <code>58</code> (middle right).


In [ ]:
disp_name("Input Part 1", inp["a"])

#### Input Part 1

.........798...145.........629....579.....455.....................13 <...view truncated>
............*.....*...........*...&...179.*........737...194........ <...>
........459..489.817........880.........*..996........*....*........ <...>
...........@.........................813............234.552..307.... <...>
...100...................*...............131..................*..... <...>
......+....#126.......214..........$......*.....+.............939... <...>
...................................854....979....177.......%........ <...>
.......427............................................683&.726...303 <...>
.....=.*...........989......888.....................$............../ <...>
..924..219...........*.....*........979............191.%.........+.. <...>
....................109........................559.....835.....708.. <...>
........................................*........=.........751...... <...>
......923...............571.....@.672....963.471.......154...*...... <...>
.........*.

In [ ]:
parts: list[int] = []
for num in remap(r=r"\d+").finditer(oneline):
    (row, lcol, rcol) = get_pos(num)
    minrow = row - 1
    maxrow = row + 1
    mincol = lcol - 1
    maxcol = rcol + 1
    for srow, scol in symbol_positions:
        if minrow <= srow <= maxrow and mincol <= scol <= maxcol:
            parts.append(int(num[0]))
            break

chk["parts"] = parts
disp_name("Wrong parts", [part for part in parts if part in chk["not_parts"]])
chk["a"] = sum(parts)

#### Parts

[798,
 145,
 629,
 579,
 455,
 130,
 243,
 154,
 179,
 737,
 194,
 854,
 560,
 699,
 459,
... <view truncated> ...


#### Wrong parts

[58, 114]

#### Part 1

520135

Every other number is adjacent to a symbol and so <em>is</em> a part number; their sum is <code><em>4361</em></code>.</p>

> <p>Of course, the actual engine schematic is much larger. <em>What is the sum of all of the part numbers in the engine schematic?</em></p>
> </article>
